Project 1

In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
#API use and web scrapping
import requests
from bs4 import BeautifulSoup
import re

In [2]:
sqlite_path = 'data/raw_data_project_m1.db'
conn_str = f'sqlite:///{sqlite_path}'
engine = create_engine(conn_str)

career_info 

In [3]:
query = 'SELECT * FROM career_info;'

In [4]:
career_info = pd.read_sql_query(query, engine)
career_info

,uuid,dem_education_level,dem_full_time_job,normalized_job_code
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,no,no,None
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,861a9b9151e11362eb3c77ca914172d0
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,no,None
3,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c
4,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f
...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,high,yes,847165cfda6b1dc82ae22b967da8af2f
9645,39f989f0-db52-0133-8482-0a81e8b09a82,high,yes,a4d5b8b38f9513825d0d94a981ebe962
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,low,no,None
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,low,yes,775190277a849cba701b306a7b374c0a


Normalized Job Titles

In [5]:
list_api = []
list_job_code = set(career_info['normalized_job_code'])

In [6]:
for i in list_job_code:
    response = requests.get(f'http://api.dataatwork.org/v1/jobs/{i}')
    result_jobs = response.json()
    list_api.append(result_jobs)
    
data_jobs = pd.DataFrame(list_api)
data_jobs

,uuid,title,normalized_job_title,parent_uuid,error
0,16b67d7f57698455aefbc1ae1b358b50,Database Engineer,database engineer,596226d5f9ef63de41a852826d483d19,NaN
1,c34f03aa0a9c6edbe8ec50f4fa3687a8,Data Center Operator,data center operator,16a750537ecaff343d0e1d9ab9f5f575,NaN
2,141ddda0f613da65a280521247de0e9f,Data Administrator,data administrator,596226d5f9ef63de41a852826d483d19,NaN
3,e605336f5a43949cb6b18054fd1937f1,Database Security Expert,database security expert,7ef30a5b2e0ec04db4ad4d5b1d354c32,NaN
4,6cf4d336112984368f83181ac5ffe948,Health Data Analyst,health data analyst,db06efbaa9cb31d42e02c047ffb0a15a,NaN
...,...,...,...,...,...
152,41b2978a2c07c5f477e60f26372d6c85,Database Management System Specialist (DBMS Sp...,database management system specialist dbms spe...,b90ca4df5690002377a7b0f1f3d40781,NaN
153,31bf3181813e21ffdfd76b5881fab066,Data Entry Supervisor,data entry supervisor,3ffe9bc56a3fd00b02fbdc71c03976f9,NaN
154,afea0e0169a98ee264db2ddb1c6e6a34,Data Processing Supervisor,data processing supervisor,3ffe9bc56a3fd00b02fbdc71c03976f9,NaN
155,2ee80cf30e2483dfe0947adc0428f8c5,Crime Data Specialist,crime data specialist,325e2251e20170928426473156bd3c48,NaN


country_info

In [7]:
query2 = 'SELECT * FROM country_info;'
country_info = pd.read_sql_query(query2, engine)
country_info.head()

,uuid,country_code,rural
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,countryside
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,city
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,Country
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,city


personal_info

In [8]:
query3 = 'SELECT * FROM personal_info;'
personal_info = pd.read_sql_query(query3, engine)

poll_info

In [9]:
query4 = 'SELECT * FROM poll_info;'
poll_info = pd.read_sql_query(query4, engine)

Web sacrapping

In [10]:
url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
html = requests.get(url).content
soup = BeautifulSoup(html, 'html.parser')
country_codes = soup.find_all('td')

In [11]:
list_country_codes =[c.text.strip().split('\n') for c in country_codes]


In [12]:
list_country_codes2 = [[l for l in sub if len(l)>1] for sub in list_country_codes]
list_country_codes3 = [l for sub in list_country_codes2 for l in sub]

In [13]:
codes_raw = []
countries= []
for i in list_country_codes3:
    if len(i)<5 or i == '(XK[1])' or i == '(CN_X_HK)':
        codes_raw.append(i)
    else:
        countries.append(i)

In [14]:
codes_str = ''.join(codes_raw)
codes = re.sub(r'[^a-zA-Z ]', ' ', codes_str).split()

In [15]:
codes[57]= 'CN_X_HK'
del codes[58]
del codes[58]

MERGED DF

In [16]:
codes_df = pd.DataFrame(codes)
countries_df = pd.DataFrame(countries)
country_codes_info = pd.concat([codes_df, countries_df.reindex(codes_df.index)], axis=1)
country_codes_info.columns = ['country_code', 'country']
country_codes_info.head()

,country_code,country
0,BE,Belgium
1,EL,Greece
2,LT,Lithuania
3,PT,Portugal
4,BG,Bulgaria


In [17]:
df_merge_countries = pd.merge(country_codes_info,
                 country_info,
                 on='country_code')
df_merge_countries = df_merge_countries[['uuid','country_code','country', 'rural']]
df_merge_countries

,uuid,country_code,country,rural
0,dcab3aa0-dc79-0133-412b-0a81e8b09a82,BE,Belgium,urban
1,2fb4da70-dcef-0133-cafd-0a81e8b09a82,BE,Belgium,urban
2,73771070-d8ec-0133-ae53-0a81e8b09a82,BE,Belgium,urban
3,3aa979b0-d93d-0133-c00b-0a81e8b09a82,BE,Belgium,rural
4,11c6e300-deb8-0133-5b60-0a81e8b09a82,BE,Belgium,urban
...,...,...,...,...
8220,a1ceab50-da22-0133-ea30-0a81e8b09a82,PL,Poland,urban
8221,74fec3c0-db5b-0133-dab4-0a81e8b09a82,PL,Poland,urban
8222,f8dc4450-da13-0133-97bb-0a81e8b09a82,PL,Poland,Non-Rural
8223,91b54950-d911-0133-c958-0a81e8b09a82,PL,Poland,city


In [18]:
data_jobs.head()
 

,uuid,title,normalized_job_title,parent_uuid,error
0,16b67d7f57698455aefbc1ae1b358b50,Database Engineer,database engineer,596226d5f9ef63de41a852826d483d19,NaN
1,c34f03aa0a9c6edbe8ec50f4fa3687a8,Data Center Operator,data center operator,16a750537ecaff343d0e1d9ab9f5f575,NaN
2,141ddda0f613da65a280521247de0e9f,Data Administrator,data administrator,596226d5f9ef63de41a852826d483d19,NaN
3,e605336f5a43949cb6b18054fd1937f1,Database Security Expert,database security expert,7ef30a5b2e0ec04db4ad4d5b1d354c32,NaN
4,6cf4d336112984368f83181ac5ffe948,Health Data Analyst,health data analyst,db06efbaa9cb31d42e02c047ffb0a15a,NaN


In [19]:
df_merge_jobs = pd.merge(data_jobs, career_info, how= 'inner', left_on= ['uuid'], right_on = ['normalized_job_code'])


In [20]:
df_merge_jobs['quantity'] = df_merge_jobs.groupby('normalized_job_code')['normalized_job_code'].transform('count')
df_merge_jobs['percentage'] = (100 * (df_merge_jobs['quantity']/df_merge_jobs['quantity'].sum())).round(6).astype(str)+'%'
df_merge_jobs

,uuid_x,title,normalized_job_title,parent_uuid,error,uuid_y,dem_education_level,dem_full_time_job,normalized_job_code,quantity,percentage
0,16b67d7f57698455aefbc1ae1b358b50,Database Engineer,database engineer,596226d5f9ef63de41a852826d483d19,NaN,896c8bd0-dcc3-0133-f1c3-0a81e8b09a82,low,yes,16b67d7f57698455aefbc1ae1b358b50,77.0,0.021821%
1,16b67d7f57698455aefbc1ae1b358b50,Database Engineer,database engineer,596226d5f9ef63de41a852826d483d19,NaN,e75d0b00-d9b1-0133-043b-0a81e8b09a82,medium,yes,16b67d7f57698455aefbc1ae1b358b50,77.0,0.021821%
2,16b67d7f57698455aefbc1ae1b358b50,Database Engineer,database engineer,596226d5f9ef63de41a852826d483d19,NaN,a60c3d40-dcaa-0133-c82f-0a81e8b09a82,high,yes,16b67d7f57698455aefbc1ae1b358b50,77.0,0.021821%
3,16b67d7f57698455aefbc1ae1b358b50,Database Engineer,database engineer,596226d5f9ef63de41a852826d483d19,NaN,cb233740-d906-0133-cee6-0a81e8b09a82,None,yes,16b67d7f57698455aefbc1ae1b358b50,77.0,0.021821%
4,16b67d7f57698455aefbc1ae1b358b50,Database Engineer,database engineer,596226d5f9ef63de41a852826d483d19,NaN,1f5c2970-da12-0133-929e-0a81e8b09a82,medium,yes,16b67d7f57698455aefbc1ae1b358b50,77.0,0.021821%
...,...,...,...,...,...,...,...,...,...,...,...
9644,d71c4164c99a5f9bca773e755a72f40f,Geographic Information Systems Database Admini...,geographic information systems database admini...,bab6b228ad976cd2624acbcfccb8ad9b,NaN,0cda45e0-dba8-0133-4f75-0a81e8b09a82,high,yes,d71c4164c99a5f9bca773e755a72f40f,73.0,0.020687%
9645,d71c4164c99a5f9bca773e755a72f40f,Geographic Information Systems Database Admini...,geographic information systems database admini...,bab6b228ad976cd2624acbcfccb8ad9b,NaN,979f0310-dd25-0133-a02b-0a81e8b09a82,medium,yes,d71c4164c99a5f9bca773e755a72f40f,73.0,0.020687%
9646,d71c4164c99a5f9bca773e755a72f40f,Geographic Information Systems Database Admini...,geographic information systems database admini...,bab6b228ad976cd2624acbcfccb8ad9b,NaN,a1747b50-d901-0133-c5e6-0a81e8b09a82,high,yes,d71c4164c99a5f9bca773e755a72f40f,73.0,0.020687%
9647,d71c4164c99a5f9bca773e755a72f40f,Geographic Information Systems Database Admini...,geographic information systems database admini...,bab6b228ad976cd2624acbcfccb8ad9b,NaN,9e9f7fe0-dd3c-0133-26f2-0a81e8b09a82,low,yes,d71c4164c99a5f9bca773e755a72f40f,73.0,0.020687%


In [21]:
df_raw = pd.merge(df_merge_countries, df_merge_jobs, left_on = 'uuid', right_on = 'uuid_y')

In [22]:
df_raw.drop(['uuid_x', 'parent_uuid', 'error', 'uuid_y', 'dem_education_level', 'dem_full_time_job', 'normalized_job_title', 'normalized_job_code'], axis = 1, inplace = True)

In [23]:
df_study = df_raw[df_raw['title'].notna()]

In [24]:
df_study

,uuid,country_code,country,rural,title,quantity,percentage
2,73771070-d8ec-0133-ae53-0a81e8b09a82,BE,Belgium,urban,Data Security Administrator,67.0,0.018987%
3,3aa979b0-d93d-0133-c00b-0a81e8b09a82,BE,Belgium,rural,Data Warehouse Developer,62.0,0.01757%
4,11c6e300-deb8-0133-5b60-0a81e8b09a82,BE,Belgium,urban,Geographic Information Systems Data Manager (G...,64.0,0.018137%
5,ed7e4bd0-d987-0133-3f9e-0a81e8b09a82,BE,Belgium,rural,Health Data Analyst,13.0,0.003684%
6,b5d05310-db4b-0133-8b2f-0a81e8b09a82,BE,Belgium,rural,Director of Data Operations,78.0,0.022104%
...,...,...,...,...,...,...,...
8216,51b2f400-d9ac-0133-2031-0a81e8b09a82,PL,Poland,urban,Database Architect,72.0,0.020404%
8217,314e9840-d982-0133-74f3-0a81e8b09a82,PL,Poland,urban,Database Administrator (DBA),10.0,0.002834%
8220,a1ceab50-da22-0133-ea30-0a81e8b09a82,PL,Poland,urban,Data Storage Specialist,83.0,0.023521%
8221,74fec3c0-db5b-0133-dab4-0a81e8b09a82,PL,Poland,urban,Automatic Data Processing Planner,78.0,0.022104%


In [25]:
df_possible_hires_raw = df_study[df_study['title'].str.match('.*Data Analyst.*|.*Data Scientist.*|.*Data Miner.*')]
df_possible_hires_raw

,uuid,country_code,country,rural,title,quantity,percentage
5,ed7e4bd0-d987-0133-3f9e-0a81e8b09a82,BE,Belgium,rural,Health Data Analyst,13.0,0.003684%
73,78156750-d949-0133-1a02-0a81e8b09a82,BE,Belgium,city,Data Scientist,67.0,0.018987%
76,5fcac6c0-dad5-0133-3d17-0a81e8b09a82,BE,Belgium,Country,Medical Data Analyst,13.0,0.003684%
81,92c27100-d9d0-0133-c4db-0a81e8b09a82,BE,Belgium,countryside,Data Scientist,67.0,0.018987%
111,917dd2a0-d955-0133-89f1-0a81e8b09a82,BE,Belgium,urban,Data Miner,72.0,0.020404%
...,...,...,...,...,...,...,...
8016,315e4cf0-d93d-0133-c007-0a81e8b09a82,PL,Poland,urban,Data Miner,72.0,0.020404%
8044,a7d665b0-def0-0133-a8ef-0a81e8b09a82,PL,Poland,rural,Medical Data Analyst,13.0,0.003684%
8069,835506b0-d9a5-0133-aa2d-0a81e8b09a82,PL,Poland,urban,Data Miner,72.0,0.020404%
8124,9de09f40-dc66-0133-baf4-0a81e8b09a82,PL,Poland,urban,Data Miner,72.0,0.020404%


In [26]:
df_possible_hires = df_possible_hires_raw[df_possible_hires_raw['rural'].str.match('rural|countryside')]


In [27]:
df_possible_hires.head()

,uuid,country_code,country,rural,title,quantity,percentage
5,ed7e4bd0-d987-0133-3f9e-0a81e8b09a82,BE,Belgium,rural,Health Data Analyst,13.0,0.003684%
81,92c27100-d9d0-0133-c4db-0a81e8b09a82,BE,Belgium,countryside,Data Scientist,67.0,0.018987%
235,0bace1e0-d94d-0133-54ac-0a81e8b09a82,PT,Portugal,rural,LiDAR Data Analyst,11.0,0.003117%
582,709bb380-d8e2-0133-b610-0a81e8b09a82,ES,Spain,countryside,Data Miner,72.0,0.020404%
1300,7593f1b0-dbec-0133-33c0-0a81e8b09a82,ES,Spain,rural,Data Miner,72.0,0.020404%


In [30]:
df_possible_hires.to_csv(r'february_amazon_rural_candidates.csv', index = False)